In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append('../src')
os.chdir('../src')

In [2]:
from raving_fader.helpers.eval import load_model,forward_eval
import IPython.display as idp
import librosa
from audio_descriptors.features import compute_all
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
from pathlib import Path


0
8


In [18]:
a = torch.load("/data/nils/datasets/nsynth_full_train/features.pth")
a[3]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [3]:
def plot_features(features,features_mod,descriptors):
    f,axs = plt.subplots(1,5,figsize=(20,5))
    for i,ax in enumerate(axs):
        ax.plot(feat[i],label="Original")
        ax.plot(feat_out[i],label="Modified")
        ax.set_ylim(-1,1)
        ax.set_title(descriptors[i])
        ax.legend()
        
def get_features(signal,pipeline,model):
    feat = compute_all(audio,sr=sr,descriptors=model.descriptors,
                                            mean=False,
                                            resample=latent_length)
    
    feat = {descr: feat[descr] for descr in model.descriptors}
    feat = np.array(list(feat.values())).astype(np.float32)
    #Normalize features
    for i,descr in enumerate(model.descriptors):                
            feat[i] = pipeline.dataset.normalize(feat[i],pipeline.dataset.min_max_features[descr]) 
    return(torch.tensor(feat))

# Model

In [6]:
models_path = "/data/nils/raving-fader/models/"
# model_name = "FRAVE_DK_multi" #Darbouka trained on 5 descr
model_name = "FRAVE_nsynthfull" #NSYNTH trained on 5 descr
# model_name = "FRAVE_multi_JPN" #Japanese trained on 5 descr


In [8]:
addon = "_900000" #To select a specific checkpoint
ckpt = os.path.join(models_path+model_name,model_name+"__vae"+addon+".ckpt")
config_file = os.path.join(models_path+model_name,"train_config.yaml")
config,pipeline,model,checkpoint=load_model(config_file,ckpt,datapath=None,batch_size=8)

>>> Device : cuda
289205
begin here


In [9]:
model

FadeRAVE(
  (pqmf): CachedPQMF(
    (forward_conv): Conv1d(1, 8, kernel_size=(257,), stride=(8,), bias=False)
    (inverse_conv): Conv1d(8, 8, kernel_size=(33,), stride=(1,), bias=False)
  )
  (loudness): Loudness()
  (encoder): Encoder(
    (net): CachedSequential(
      (0): Conv1d(8, 64, kernel_size=(7,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
      (3): Conv1d(64, 128, kernel_size=(9,), stride=(4,))
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.2)
      (6): Conv1d(128, 256, kernel_size=(9,), stride=(4,))
      (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): LeakyReLU(negative_slope=0.2)
      (9): Conv1d(256, 512, kernel_size=(9,), stride=(4,))
      (10): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11)

# Audio

Gets a sample from the training set (cropped audio)

In [ ]:
audio = pipeline.dataset.env[0] #GET A SAMPLE FROM TRAINING SET
idp.Audio(audio,rate = config.data.sr)

Raw dataset paths

#### Nsynth

In [ ]:
path = "/data/nils/nsynth_string/audio"
sr = 16000
files = os.listdir(path)
i = random.sample(range(len(files)),1)[0]
audio,sr = librosa.load(os.path.join(path,files[i]),sr=sr)
audio = np.pad(audio,(0,2**16-len(audio)))

idp.Audio(audio,rate = sr)

#### Japanese voice

In [ ]:
path = "/data/nils/japanese/audio/voiceactress100/wav"
sr = 48000
files = os.listdir(path)
audio,sr = librosa.load(os.path.join(path,files[0]),sr=sr)
idp.Audio(audio,rate = sr)

#### Darbouka

In [ ]:
path = "/data/nils/darbouka/darbouka_Loops/"
extension = ["*.wav"]
wavs=[]
for f, folder in enumerate([path+l for l in os.listdir(path)]):
    print("Recursive search in {}".format(folder))
    for ext in extension:
        wavs.extend(list(Path(folder).rglob(ext)))
i = random.sample(range(len(wavs)),1)[0]
file = wavs[i]
audio,sr = librosa.load(file,sr=sr)
audio = np.pad(audio,(0,2**19-len(audio)))
idp.Audio(audio,rate = sr)

In [302]:
a = torch.randn(4)
b= torch.randn(4)
crit = torch.nn.MSELoss(reduction='none')
crit(a,b)

tensor([3.7798, 0.1582, 1.9149, 0.5391])

### Original

In [ ]:
latent_length = len(audio) // (pipeline.model_config.data_size * np.prod(pipeline.model_config.ratios))
feat = get_features(audio,pipeline,model)

In [ ]:
target_descr = [2]

t = np.linspace(0,latent_length,latent_length)
mod1 = 0.2*np.sin(t*100)

mod2 = 1*np.linspace(-1,1,latent_length)

mods = [mod2]


feat_out = feat.clone()

for i,index in enumerate(target_descr):
    feat_out[index] = torch.tensor(mods[i])+  feat_out[index]

plot_features(feat,feat_out,model.descriptors)
y = forward_eval(model,torch.tensor(audio).unsqueeze(0),torch.tensor(feat_out).unsqueeze(0)).cpu().numpy()[0]
idp.Audio(y,rate = sr)